In [2]:
import pandas as pd
import numpy as np
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
train = pd.read_csv('train_tweets.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
x = train["text"]
y = train["target"]

In [10]:
def preprocess_text(text):
    stpword = stopwords.words('english') # and, the, it, 
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    
    no_stopwords = ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])
    
    #I like food
    #['I', 'like', 'food']
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(no_stopwords)
    
    
    stemmer = nltk.stem.PorterStemmer()
    stemmed = " ".join(stemmer.stem(token) for token in tokens)
    
    return stemmed
    
x = x.apply(preprocess_text)
x.head()

0            deed reason earthquak may allah forgiv us
1                 forest fire near La rong sask canada
2    resid ask shelter place notifi offic evacu she...
3    13000 peopl receiv wildfir evacu order california
4    got sent photo rubi alaska smoke wildfir pour ...
Name: text, dtype: object

In [9]:
# stemming example
text = "feet cats wolves talked"

tokenizer = nltk.tokenize.WhitespaceTokenizer()
tokens = tokenizer.tokenize(text)

stemmer = nltk.stem.PorterStemmer()
print("Stemming: ", " ".join(stemmer.stem(token) for token in tokens))


Stemming:  feet cat wolv talk


In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=32)
ytrain = ytrain.values
ytest = ytest.values
prediction_tests_texts = pd.DataFrame(data = {"text":xtest[:15],  "target":ytest[:15]})

In [13]:
vect = CountVectorizer()
xtrain = vect.fit_transform(xtrain)
xtest = vect.transform(xtest)
tfidf = TfidfTransformer()
xtrain = tfidf.fit_transform(xtrain)
xtest = tfidf.transform(xtest)
xtrain = xtrain.toarray()
xtest = xtest.toarray()



In [12]:
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']
vect = CountVectorizer()
vect.fit(simple_train)
print(vect.get_feature_names())
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm.toarray()

['cab', 'call', 'me', 'please', 'tonight', 'you']


array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [14]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(xtrain, ytrain)
model.score(xtest, ytest)

0.8057742782152231

In [15]:
prediction_tests_texts["predicted"] =  model.predict(xtest[:15])
prediction_tests_texts.head(15)

,text,target,predicted
2250,delug low selfimag take quiz httptcoxspqdoriqj...,1,0
7155,bedroom clean bathroom clean laundri done shit...,0,0
3148,updat indiana state polic reopen i65 near lafa...,1,1
1857,minut alon crushit overr experiencesmh,0,1
4760,lightn caus six new fire vancouveråêisland htt...,1,1
1416,anoth day pass thank central command confirm n...,1,1
3844,ad fire flame saturday rick wonder spin guest ...,0,1
2715,man currensi realli talkin talk Id devast ghos...,1,0
2012,new post blog httptcoavu9b4k2rv thesensualey m...,0,0
3282,littl studio middl nowher becom epicentr commu...,0,0
